### webscraper to download membership data

In [1]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs

In [2]:
# initialize browser instance
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# visit url
url = 'https://www.citibikenyc.com/system-data/'
browser.visit(url)

In [4]:
# turn into soup
html = browser.html
soup = bs(html,'html.parser')

In [5]:
# find all unordered lists
lists = soup.find_all('ul')

In [6]:
# find all bullets
items = lists[4].find_all('li')

In [7]:
# grab all hyperlinks
urls = []
for item in items:
    for link in item.find_all('a'):
        urls.append(link['href'])

In [8]:
# confirm 22 hyperlinks found
len(urls)

22

In [9]:
# check for data sources
exists = []
for dest in urls:
    browser.visit(dest)
    exists.append(browser.is_text_present('Get the data', wait_time=None))

In [10]:
# confirm 22 sources found
print(f'Data sources found: {exists.count(True)}')
print(f'Data sources not found: {exists.count(False)}')

Data sources found: 22
Data sources not found: 0


### download each CSV file and rename

In [41]:
import time
import os

In [12]:
# generate contextual labels
labels = []
for item in items:
    year = item.text[:4]
    for link in item.find_all('a'):
        span = link.text.replace(' ','').replace(';','')
        labels.append(year + '-' + span + '.csv')

In [13]:
# change directory to downloads
path = 'C:/Users/nssas/Downloads/'
os.chdir(path)

# visit each source and download on click
dest = 'D:/tableau_csvs/membership_data/'
for i, url in enumerate(urls):
    browser.visit(url)
    browser.click_link_by_partial_text('Get the data')
    
    time.sleep(1)

    # rename file inplace, move to resources directory
    for file in os.listdir('.'):
        if file.startswith('data-'):
            os.rename(file,dest+labels[i])     

In [14]:
# navigate back to project directory
nav = 'D:/tableau_csvs/'
os.chdir(nav)

In [15]:
# confirm files are renamed
for file in os.listdir('membership_data'):
    print(file)

2013-Launch-Sep.csv
2013-Oct-Dec.csv
2014-Apr-Jun.csv
2014-Jan-Mar.csv
2014-Jul-Sep.csv
2014-Oct-Dec.csv
2015-Apr-Jun.csv
2015-Jan-Mar.csv
2015-Jul-Sep.csv
2015-Oct-Dec.csv
2016-Apr-Jun.csv
2016-Jan-Mar.csv
2016-Jul-Sep.csv
2016-Oct-Dec.csv
2017-Apr-Jun.csv
2017-Jan-Mar.csv
2017-Jul-Sep.csv
2017-Oct-Dec.csv
2018-Apr-Jun.csv
2018-Jan-Mar.csv
2018-Jul-Sep.csv
2018-Oct-Dec.csv


In [16]:
# close browser instance
browser.quit()

### cut down data

In [2]:
import pandas as pd

In [19]:
# check difference in data columns
for file in os.listdir('membership_data'):
    df = pd.read_csv(f'membership_data/{file}')
    print(len(df.columns))

9
8
9
9
9
9
8
8
8
8
9
8
8
8
6
6
6
6
6
6
6
6


In [20]:
unique = []
for file in os.listdir('membership_data/'):
    df = pd.read_csv(f'membership_data/{file}')
    cols = df.columns.values.tolist()
    for col in cols:
        if col not in unique:
            unique.append(col)

In [21]:
# there's no data for the last tabbed 7-day, so just using .startswith()len
unique

['Date',
 'Trips over the past 24-hours (midnight to 11:59pm)',
 'Cumulative trips (since launch):',
 'Miles traveled today (midnight to 11:59 pm)',
 'Miles traveled to date:',
 'Total Annual Members',
 'Annual Member Sign-Ups (midnight to 11:59 pm)',
 '24-Hour Passes Purchased (midnight to 11:59 pm)',
 '7-Day Passes Purchased (midnight to 11:59 pm)',
 '24-Hour Passes Purchased (midnight to 11:59 pm)\t7-Day Passes Purchased (midnight to 11:59 pm)',
 'Total Annual Memberships Sold',
 'Cumulative trips (since launch)',
 'Miles traveled to date',
 'Total Annual Members (All Time)',
 '3-Day Passes Purchased (midnight to 11:59 pm)']

In [22]:
a = []
b = []
c = []

for file in os.listdir('membership_data'):
    df = pd.read_csv(f'membership_data/{file}')
    cols = df.columns.tolist()
    for col in cols:
        if col.startswith('3-Day Passes'):
            a.append(col)
        if col.startswith('7-Day Passes'):
            b.append(col)
        if col.startswith('24-Hour Passes'):
            c.append(col)
            
print(f'CSVs with 3-day passes: {len(a)}')
print(f'CSVs with 7-day passes: {len(b)}')
print(f'CSVs with 24-hr passes: {len(c)}')

CSVs with 3-day passes: 11
CSVs with 7-day passes: 11
CSVs with 24-hr passes: 22


#### notes for me

constant:
* Date
* Trips over the past 24-hours (midnight to 11:59pm)
* Miles traveled today
* Total Annual Members/Memberships Sold
* 24-Hour Passes Purchased
* 11 7-Day, 11 3-Day

to do:
* combine '24-Hour Passes'; no data for the last tabbed 7-day, so just using .startswith()
* combine 'Miles traveled to date'
* no need for cumulative trips, just count totals
* 7 day passes end in 2016, switch to 3 day passes
* missing 7-day passes for 2013-oct-dec
* both passes available for 2016-apr-jun


In [24]:
keep = ['Date',
        'Trips over the past 24-hours',
        'Miles traveled today',
        'Total Annual Members',
        '24-Hour Passes',
        '7-Day Passes',
        '3-Day Passes']

In [25]:
# all should be 6 except for 2013-oct-dec (no day passes)
# and 2016-apr-jun (3 and 7-day)
for file in os.listdir('membership_data/'):
    df = pd.read_csv(f'membership_data/{file}')
    holder = []
    for col in df.columns:
        for item in keep:
            if col.startswith(item):
                holder.append(col)

    dic = {}
    for i, col in enumerate(holder):
        if col.startswith('3'):
            dic[col] = keep[-1]
        elif col.startswith('7'):
            dic[col] = keep[-2]
        else:
            dic[col] = keep[i]

    print(f'Columns in {file}: {len(dic)}')
    df[holder].rename(index=str,columns=dic).to_csv(f'membership_data/{file}',index=False)

Columns in 2013-Launch-Sep.csv: 6
Columns in 2013-Oct-Dec.csv: 5
Columns in 2014-Apr-Jun.csv: 6
Columns in 2014-Jan-Mar.csv: 6
Columns in 2014-Jul-Sep.csv: 6
Columns in 2014-Oct-Dec.csv: 6
Columns in 2015-Apr-Jun.csv: 6
Columns in 2015-Jan-Mar.csv: 6
Columns in 2015-Jul-Sep.csv: 6
Columns in 2015-Oct-Dec.csv: 6
Columns in 2016-Apr-Jun.csv: 7
Columns in 2016-Jan-Mar.csv: 6
Columns in 2016-Jul-Sep.csv: 6
Columns in 2016-Oct-Dec.csv: 6
Columns in 2017-Apr-Jun.csv: 6
Columns in 2017-Jan-Mar.csv: 6
Columns in 2017-Jul-Sep.csv: 6
Columns in 2017-Oct-Dec.csv: 6
Columns in 2018-Apr-Jun.csv: 6
Columns in 2018-Jan-Mar.csv: 6
Columns in 2018-Jul-Sep.csv: 6
Columns in 2018-Oct-Dec.csv: 6


In [27]:
dic

{'Date': 'Date',
 'Trips over the past 24-hours (midnight to 11:59pm)': 'Trips over the past 24-hours',
 'Miles traveled today (midnight to 11:59 pm)': 'Miles traveled today',
 'Total Annual Members (All Time)': 'Total Annual Members',
 '24-Hour Passes Purchased (midnight to 11:59 pm)': '24-Hour Passes',
 '3-Day Passes Purchased (midnight to 11:59 pm)': '3-Day Passes'}

In [32]:
# find files of interest by year
for x in range(2013,2019):
    for file in os.listdir('membership_data'):
        if file.startswith(str(x)):
            print(file)
            
    print('----')

2013-Launch-Sep.csv
2013-Oct-Dec.csv
----
2014-Apr-Jun.csv
2014-Jan-Mar.csv
2014-Jul-Sep.csv
2014-Oct-Dec.csv
----
2015-Apr-Jun.csv
2015-Jan-Mar.csv
2015-Jul-Sep.csv
2015-Oct-Dec.csv
----
2016-Apr-Jun.csv
2016-Jan-Mar.csv
2016-Jul-Sep.csv
2016-Oct-Dec.csv
----
2017-Apr-Jun.csv
2017-Jan-Mar.csv
2017-Jul-Sep.csv
2017-Oct-Dec.csv
----
2018-Apr-Jun.csv
2018-Jan-Mar.csv
2018-Jul-Sep.csv
2018-Oct-Dec.csv
----


In [142]:
# combine quarterly data for annual data
for x in range(2013,2019):
    quarters = []
    for file in os.listdir('membership_data/'):
        if file.startswith(str(x)):
            df = pd.read_csv(f'membership_data/{file}')
            quarters.append(df)

    # combine and sort by date, ensure type
    year_df = pd.concat(quarters,ignore_index=True,sort=False)
    year_df['Date'] = pd.to_datetime(year_df.Date)
    year_df = year_df.sort_values(by='Date')

    year_df.to_csv(f'membership_data/output/bulk_{x}.csv',sep=',',index=False)
            
    print(f'Created bulk CSV for {x}...')


Created bulk CSV for 2013...
Created bulk CSV for 2014...
Created bulk CSV for 2015...
Created bulk CSV for 2016...
Created bulk CSV for 2017...
Created bulk CSV for 2018...
